In [7]:
from Paramaters.Parameter import Parameter
import os
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [8]:
window_size = 15
predict_monthes=3
dataset_path = "C:\\Users\\Yuxia\\Desktop\\CS-Project\\T-BiLSTM\\code-test\\Project_VStart\\data\\test\\encoded_MonthlyData.csv"
dataset = pd.read_csv(dataset_path, encoding='utf-8')

In [9]:
X_slides = []
Y_slides = []
grouped = dataset.groupby('LOAN SEQUENCE NUMBER')
for name, group in tqdm(grouped, desc="Processing groups"):
    if len(group) >= window_size + predict_monthes:
        group_size = len(group)
        group = group.sort_values('REMAINING MONTHS TO LEGAL MATURITY', ascending=False)
        for i in range(group_size - window_size - predict_monthes + 1):
            X_slides.append(group.iloc[i:i + window_size])
            Y_slides.append(group.iloc[i + window_size:i + window_size + predict_monthes])

Processing groups: 100%|██████████| 1447/1447 [00:02<00:00, 520.91it/s]


In [10]:
X1 = X_slides[0]
Y1 = Y_slides[0]

In [4]:
# 对于所有在X_slides中的X, 要去除：'LOAN SEQUENCE NUMBER'， 'MONTHLY REPORTING PERIOD', 'CURRENT LOAN DELINQUENCY STATUS

for i in tqdm(range(len(X_slides)), desc="Processing X_Slides"):
    X_slides[i] = X_slides[i].drop(columns=['LOAN SEQUENCE NUMBER', 'MONTHLY REPORTING PERIOD', 'CURRENT LOAN DELINQUENCY STATUS'])
    X_slides[i] = X_slides[i].to_numpy()
    X_slides[i] = X_slides[i].astype(np.float32)

Processing X_Slides: 100%|██████████| 742487/742487 [01:48<00:00, 6849.17it/s]


In [7]:
# 对于所有在Y_slides中的Y,只要一个y_list
y_list = []
for i in tqdm(range(len(Y_slides)), desc="Processing y_Slides"):
    result = 0
    for j in range(3):
        result += Y_slides[i]["CURRENT LOAN DELINQUENCY STATUS"].iloc[j]
    if result > 0:
        y_list.append(1)
    else:
        y_list.append(0)

Processing y_Slides: 100%|██████████| 742487/742487 [00:22<00:00, 33679.42it/s]


In [8]:
# 数据重采样Undersampling
X_array = np.array(X_slides)
y_array = np.array(y_list)
idx_0 = np.where(y_array == 0)[0]
idx_1 = np.where(y_array == 1)[0]
n_samples = min(len(idx_0), len(idx_1))
np.random.seed(42)  # 设置随机种子以确保可重复性
# 随机选择idx_0和idx_1中的样本, 不放回抽样, 数量为n_samples
idx_0_sampled = np.random.choice(idx_0, size=n_samples, replace=False)
idx_1_sampled = np.random.choice(idx_1, size=n_samples, replace=False)
idx_balanced = np.concatenate([idx_0_sampled, idx_1_sampled])
np.random.shuffle(idx_balanced)
X_balanced = X_array[idx_balanced]
y_balanced = y_array[idx_balanced]
print("X_balanced.shape:", X_balanced.shape)
print("y_balanced.shape:", y_balanced.shape)

X_balanced.shape: (14424, 15, 22)
y_balanced.shape: (14424,)


In [9]:
# 进行训练集和测试集划分， 70%训练集，30%测试集， 随机种子42， 打乱数据， stratify= y_balanced
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42, shuffle=True, stratify=y_balanced)

In [10]:
store_folder = "C:\\Users\\Yuxia\\Desktop\\CS-Project\\T-BiLSTM\\code-test\\Project_VStart\\data\\test"
np.save(os.path.join(store_folder, "trainX.npy"), X_train)
np.save(os.path.join(store_folder, "testX.npy"), X_test)
np.save(os.path.join(store_folder, "trainY.npy"), y_train)
np.save(os.path.join(store_folder, "testY.npy"), y_test)